In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import time
from datetime import datetime, timedelta
from selenium import webdriver
import requests
from selenium.webdriver.common.keys import Keys
import re
from contextlib import closing
import os

In [2]:
#Change the working directory to the Downloads folder, since we need to download a file and use it here
os.chdir(r'C:\Users\anand\Downloads')
path = os.getcwd()
print(path)

C:\Users\anand\Downloads


In [3]:
#Defining the path for chrome driver to be used for selenium
execute_path = '/Users/anand/Downloads/chromedriver'

In [4]:
#Web scraping coinmarketcap website to look for top performing cryptocurrencies

url = "https://coinmarketcap.com"

#Since the website has javascript, using selenium to load it first and then saving html
driver = webdriver.Chrome(executable_path=execute_path)
driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(10)

driver.get(url)
driver.maximize_window()
for i in range(5):
    coin_name = driver.find_elements_by_xpath('//*[@id="__next"]/div[1]/div[1]/div[2]/div/div[3]/div[2]/div/p[1]/button')
    
    html = driver.find_element_by_tag_name('html')
    html.send_keys(Keys.PAGE_DOWN)

with open('crypto_home.htm', 'w',encoding="utf-8") as file:
    file.write(driver.page_source)

<ipython-input-4-a81c8ca964d1>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=execute_path)
<ipython-input-4-a81c8ca964d1>:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  coin_name = driver.find_elements_by_xpath('//*[@id="__next"]/div[1]/div[1]/div[2]/div/div[3]/div[2]/div/p[1]/button')
<ipython-input-4-a81c8ca964d1>:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  html = driver.find_element_by_tag_name('html')


In [5]:
#loading the html file saved in the last step
file = open("crypto_home.htm", "r",encoding="utf-8")
mainpage = file.read()
soup = BeautifulSoup(mainpage, 'lxml')

In [6]:
#Getting the list of top performing cryptocurrencies from the website
Coins = []

for i in range(13):
    temp = re.compile("([a-zA-Z]+)([0-9]+)")
    try:
        x = soup.find('tbody').find_all('td')[i*11 + 2].text.strip()
        res = temp.match(x).groups()
        Coins.append(str(res[0]))
    except:
        continue

In [7]:
#Web scraping coinmarketcap website to look for top performing cryptocurrencies

url = "https://www.alphavantage.co/digital_currency_list/"

#Since the website has javascript, using selenium to load it first and then saving html
driver = webdriver.Chrome(executable_path=execute_path)
driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(10)

driver.get(url)

<ipython-input-7-5760324f2875>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=execute_path)


In [8]:
#Using the mapping provided by alphavantage to map currencies with their symbols
currency_code = pd.read_csv("digital_currency_list.csv")

In [9]:
#Getting the currency symbols in the list "currency"
currency = []

for i in Coins:
    try:
        x = currency_code.loc[currency_code['currency name'] == i]['currency code'].values
        x = str(x).strip('[]')
        currency.append(x)
    except:
        continue

currency = [item.replace(r"'", '') for item in currency]
currency = [i for i in currency if i]

#Creating a function to use alpha vantage api

data = []

def get_crypto_price(symbol, exchange, start_date = None):
    api_key = '0Q0SMB02VX0CITY5'
    api_url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market={exchange}&apikey={api_key}'
    raw_df = requests.get(api_url).json()
    df = pd.DataFrame(raw_df['Time Series (Digital Currency Daily)']).T
    df = df.rename(columns = {'1a. open (USD)': 'open', '2a. high (USD)': 'high', '3a. low (USD)': 'low', '4a. close (USD)': 'close', '5. volume': 'volume'})
    for i in df.columns:
        df[i] = df[i].astype(float)
    df.index = pd.to_datetime(df.index)
    df = df.iloc[::-1].drop(['1b. open (USD)', '2b. high (USD)', '3b. low (USD)', '4b. close (USD)', '6. market cap (USD)'], axis = 1)
    df.insert(0, 'Currency', symbol)
    if start_date:
        df = df[df.index >= start_date]
    return df

for symbol in currency:
    try:
        if len(data) == 0:
            data = get_crypto_price(symbol = symbol, exchange = 'USD', start_date = '2020-01-01')
        else:
            data = data.append(get_crypto_price(symbol = symbol, exchange = 'USD', start_date = '2020-01-01'))
    except:
        continue

In [10]:
#Market data from alpha vantage
data.reset_index(inplace=True)
data.rename({'index': 'Date'}, axis=1, inplace=True)
data

,Date,Currency,open,high,low,close,volume
0,2020-01-01,BTC,7195.240,7255.000,7175.150,7200.850,1.679239e+04
1,2020-01-02,BTC,7200.770,7212.500,6924.740,6965.710,3.195148e+04
2,2020-01-03,BTC,6965.490,7405.000,6871.040,7344.960,6.842850e+04
3,2020-01-04,BTC,7345.000,7404.000,7272.210,7354.110,2.998797e+04
4,2020-01-05,BTC,7354.190,7495.000,7318.000,7358.750,3.833109e+04
...,...,...,...,...,...,...,...
2998,2022-03-15,ADA,0.803,0.817,0.781,0.800,9.546221e+07
2999,2022-03-16,ADA,0.800,0.840,0.792,0.838,1.607970e+08
3000,2022-03-17,ADA,0.839,0.858,0.827,0.835,1.064629e+08
3001,2022-03-18,ADA,0.835,0.860,0.820,0.851,8.286573e+07


In [11]:
#Joining this table with the mapping list we created earlier
x = pd.merge(data, currency_code, left_on='Currency', right_on='currency code')
x = x.drop('currency code', 1)
x

,Date,Currency,open,high,low,close,volume,currency name
0,2020-01-01,BTC,7195.240,7255.000,7175.150,7200.850,1.679239e+04,Bitcoin
1,2020-01-02,BTC,7200.770,7212.500,6924.740,6965.710,3.195148e+04,Bitcoin
2,2020-01-03,BTC,6965.490,7405.000,6871.040,7344.960,6.842850e+04,Bitcoin
3,2020-01-04,BTC,7345.000,7404.000,7272.210,7354.110,2.998797e+04,Bitcoin
4,2020-01-05,BTC,7354.190,7495.000,7318.000,7358.750,3.833109e+04,Bitcoin
...,...,...,...,...,...,...,...,...
2998,2022-03-15,ADA,0.803,0.817,0.781,0.800,9.546221e+07,Cardano
2999,2022-03-16,ADA,0.800,0.840,0.792,0.838,1.607970e+08,Cardano
3000,2022-03-17,ADA,0.839,0.858,0.827,0.835,1.064629e+08,Cardano
3001,2022-03-18,ADA,0.835,0.860,0.820,0.851,8.286573e+07,Cardano


In [12]:
#Creating a unique list of keywords from the previous step
keywords = pd.unique(x['currency name'])
keywords = keywords.tolist()

In [13]:
#Defining the token provided by Twitter API
TOKEN = #mention your token
def auth():
    return TOKEN

In [14]:
#Defining functions in the next 3 steps to be used to get aggregated historical data from twitter
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [15]:
#Defining a function to include the API url for getting aggregate data on tweets
def create_url(keyword, start_date, end_date, granularity, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/counts/all"

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'granularity': granularity,
                    'next_token': {}
                   }
    return (search_url, query_params)

In [16]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [17]:
#Defining values for metrics in functions above
bearer_token = auth()
headers = create_headers(bearer_token)
start_date = "2020-01-01T00:00:00.000Z"
start_date_date = datetime.strptime(start_date, '%Y-%m-%dT00:00:00.000Z')
end_date_date = datetime.now()
end_date = end_date_date.strftime('%Y-%m-%dT00:00:00.000Z')
granularity = "day"
max_results = 45
df = []

In [18]:
#Running the functions defined above for all the currencies in the table
for keyword in keywords:
    while (end_date_date > start_date_date):
        url = create_url(keyword, start_date, end_date, granularity, max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1])
        if len(df) == 0:
            df = pd.DataFrame(json_response["data"])
            df.insert(0, 'Keywords', keyword)
        else:
            df_new = pd.DataFrame(json_response["data"])
            df_new.insert(0, 'Keywords', keyword)
            df = df.append(df_new)
        end_date_date = datetime.strptime(end_date, '%Y-%m-%dT00:00:00.000Z') - timedelta(days=31)
        end_date = end_date_date.strftime('%Y-%m-%dT00:00:00.000Z')
    end_date_date = datetime.now()
    end_date = end_date_date.strftime('%Y-%m-%dT00:00:00.000Z')

Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Co

In [19]:
#Changing the date format in the table
df.start = [datetime.strptime(x, '%Y-%m-%dT00:00:00.000Z') for x in df.start]
df.end = [datetime.strptime(x, '%Y-%m-%dT00:00:00.000Z') for x in df.end]
df

,Keywords,end,start,tweet_count
0,Bitcoin,2022-02-16,2022-02-15,531596
1,Bitcoin,2022-02-17,2022-02-16,504114
2,Bitcoin,2022-02-18,2022-02-17,499940
3,Bitcoin,2022-02-19,2022-02-18,469294
4,Bitcoin,2022-02-20,2022-02-19,434730
...,...,...,...,...
27,Cardano,2020-01-30,2020-01-29,1941
28,Cardano,2020-01-31,2020-01-30,1021
29,Cardano,2020-02-01,2020-01-31,1075
30,Cardano,2020-02-02,2020-02-01,1058


In [20]:
#Merging the market value table with the tweet count table
final_table = pd.merge(x, df, left_on=['Date','currency name'], right_on=['start','Keywords'])
final_table = final_table.drop(['Keywords','end','start'], 1)
final_table

,Date,Currency,open,high,low,close,volume,currency name,tweet_count
0,2020-01-01,BTC,7195.240,7255.000,7175.150,7200.850,1.679239e+04,Bitcoin,33182
1,2020-01-02,BTC,7200.770,7212.500,6924.740,6965.710,3.195148e+04,Bitcoin,39966
2,2020-01-03,BTC,6965.490,7405.000,6871.040,7344.960,6.842850e+04,Bitcoin,45382
3,2020-01-04,BTC,7345.000,7404.000,7272.210,7354.110,2.998797e+04,Bitcoin,36885
4,2020-01-05,BTC,7354.190,7495.000,7318.000,7358.750,3.833109e+04,Bitcoin,35443
...,...,...,...,...,...,...,...,...,...
2990,2022-03-13,ADA,0.789,0.821,0.780,0.786,8.873369e+07,Cardano,39983
2991,2022-03-14,ADA,0.787,0.816,0.776,0.802,1.041420e+08,Cardano,35504
2992,2022-03-15,ADA,0.803,0.817,0.781,0.800,9.546221e+07,Cardano,33645
2993,2022-03-16,ADA,0.800,0.840,0.792,0.838,1.607970e+08,Cardano,37080


In [29]:
from pymongo import MongoClient

#Connecting to MongoDB and creating a database
client = MongoClient('mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&directConnection=true&ssl=false')
Crypto_market = client['Crypto_market']
Crypto_market_daily = Crypto_market['Crypto_market_daily']

#Inserting the data into sf_donut_shops
Crypto_market_daily.insert_many(final_table.to_dict('records'))